In [ ]:
import nltk
from nltk.corpus import gutenberg
from nltk.corpus import stopwords

In [ ]:
nltk.data.path.append("nltk_data")

In [ ]:
porter = nltk.PorterStemmer()

def sum_common_words(common_words):
    s=0
    for w in common_words:
        s=s+w[1]
    return s

def define_sentence_interest(common_words,sum_words_count,sents): #Define sentence interestingness
    sent_interest=[0]*len(sents)
    for idx,sent in enumerate(sents) :
        for w in sent :
            for cw in common_words:
                if porter.stem(w.lower()) == cw[0]:   
                    sent_interest[idx]+=cw[1]/sum_words_count #use the proportion as 'weight'
    return sent_interest

def key(item):
    return item[2]

def filter_sentences(threshold,sent_interest,sents): #Filter sentences based on their length and interestingness
    interest_text=[]
    for idx,s in enumerate(sent_interest):
        if sent_interest[idx]>threshold and len(sents[idx])>4 and len(sents[idx])<max_sentence_length: #if 1/4 of the words in the sentece contain words from the most common words 
                                            #then consider this sentece as valuable
            interest_text.append((idx,sents[idx],sent_interest[idx]))
    return interest_text

def generate_text(sorted_list): #Generate the summary 
    j=0
    i=0
    summary_text=''
    while i < summary_words:
        if j> len(sorted_list) : break
        sentence=sorted_list[j][1]    
        j+=1
        summary_text+="\n * "
        for w in sentence:
            #print(w)
            summary_text+=w+' '
            i+=1;
            if i>=summary_words : break
    return summary_text

In [ ]:
def summarize(text):

    sents = gutenberg.sents(text)
    words = gutenberg.words(text)
    distinct_words=set(words)
    stop_words=stopwords.words('english')
    filtered_words = [porter.stem(w.lower()) for w in words if w.lower() not in stop_words and len(w)>4]
 
    fdist=nltk.FreqDist(filtered_words)
    common_words=fdist.most_common(20);
    sent_interest=[0]*len(sents)
    sum_words_count=0
    sum_words_count=sum_common_words(common_words) #use this in order to determine the 'weight'
                                                    #of the word from the common words list
                                                    #the higher number of occurences -> the bigger the 'weight'

    sent_interest=define_sentence_interest(common_words,sum_words_count,sents)

    threshold = sorted(common_words,reverse=False)[0][1]/sum_words_count
    
    interest_text=filter_sentences(threshold,sent_interest,sents)  

    sorted_list=list(sorted(interest_text,key= key,reverse=True))
    #print(len(sorted_list))
    summary_text=generate_text(sorted_list)
    print(summary_text)

In [ ]:
gutenberg.fileids()

In [ ]:
text = 'carroll-alice.txt' #input the fileid from the Gutenberg corpus

summary_words=200
max_sentence_length=50 #The maximum number of words in a sentence

summarize(text)